In [ ]:
import db
from importlib import reload
db = reload(db)
db = db.DB()

In [ ]:
firstNotNone = lambda it: next((x for x in it if x is not None), None)    

In [ ]:
identifiers = ('doi', 'nnt', 'ensam', 'ird', 'irstea', 'oatao', 'pubmed', 'arxiv', 'sciencespo', 'hal')

def getids(d):
    ids = []
    def append(k, v):
        if type(v) is list:
            ids.extend((k, item) for item in v)
        else:
            ids.append((k, v))
            
    for k, v in d.items():
        if k.endswith('Id_s'):
            append(k[:-4], v)
        elif k.endswith('_title_s'):
            append(k[:-2], v)
    
    if 'citationRef_s' in d:
        append('cite', d['citationRef_s'])
            
    pstart, pend = get_pages(d.get('page_s'))
    if pstart:     
        if 'journalTitle_s' in d and 'issue_s' in d:
            append('journal', [dict(title=d['journalTitle_s'], issue=issue, pstart=pstart, pend=pend) 
                               for issue in d['issue_s']])
            
        bookTitle = d.get('conferenceTitle_s') or d.get('bookTitle_s')
        if bookTitle or 'isbn_s' in d:
            append('book', dict(title=bookTitle, isbn=d.get('isbn_s'), pstart=pstart, pend=pend))
    return ids



In [ ]:
import json
from urllib import request
from urllib.parse import urlencode
from datetime import datetime

#query = {'authFullName_t':'Pierre Peterlongo'}
query = {'authFullName_t':'Olivier Dameron'}
fields = ['authFullName_s', 'authOrganism_s', 'authQuality_s', 'producedDate_tdate',
          '*_abstract_s', 'files_s', 'language_s',
          'keyword_s',
          # Identification
          '*_title_s', '*Id_s', 'isbn_s', 'bookTitle_s', 'docType_s',
          # Journal
          'journalTitle_s', 'issue_s', 'page_s', 'conferenceTitle_s',
          # Classifications
          'classification_s', 'domain_s', 'acm_s', 'jel_s', 'mesh_s', 
          'labStructAcronym_s', 'structName_s', 'authOrganism_s', 'collaboration_s',
          'citationRef_s'
         ]
params = dict(
    q= ' && '.join(field+':'+ value for field, value in query.items()),
    fl= ','.join(fields),
    wt= 'json',
    rows= 10000,
)
params = urlencode(params)
r = db.http_cache.get('https://api.archives-ouvertes.fr/search/?' + params)
r = json.loads(r.decode('utf-8'))

for d in r['response']['docs']:
    ids = getids(d)
    #for k,v in ids:
        #print("%s:\t%r" % (k,v))
    #print()
    
    print()
#     print(ids)
    doc_id = docid_byref(con,ids)
    if doc_id is None:
        doc_id = db.docTable.insert(date=datetime.strptime(d['producedDate_tdate'], "%Y-%m-%dT%H:%M:%Sz" ),
                                    type=d['docType_s'],
                                    return_id=True)
        for reftype, ref in ids:
            db.insert_ref(doc_id, reftype, ref)


#     if 'journalTitle_s' in d or 'issue_s' in d or 'isbn_s' in d or 'page_s' in d or 'bookTitle_s' in d:
#         #print(d['journalTitle_s'], d['issue_s'], d['page_s'])
#         for k, v in d.items():
#             print('%s: %s\n' % (k,v))
#     else:
#         print(d['docType_s'], d['title_s'])
#     print('-'*100)

con.commit()

In [ ]:
from private import elsevier_key
def quick_get(url):
    headers={'Accept':'application/json',
             'X-ELS-APIKey': elsevier_key}
    return json.loads(db.http_cache.get(url, headers=headers).decode('utf-8'))

In [ ]:
quick_get('http://api.crossref.org/works?query.author=olivier+dameron')['message']['items']

In [ ]:
url = ("http://api.elsevier.com/content/abstract/doi/"
          + '10.1016/j.ics.2005.03.213'
          + "?field=authors,title,publicationName,volume,issueIdentifier,item,"
          + "prism:pageRange,coverDate,article-number,doi,citedby-count,prism:aggregationType,dc:description"
      )
quick_get(url)

In [ ]:
quick_get('http://api.elsevier.com/content/search/author?' + urlencode(dict(query='authfirst(Olivier) AND authlast(Dameron)')) )

In [ ]:
quick_get('http://api.crossref.org/works/10.1016/j.ics.2005.03.213')

In [ ]:
con.commit()
# con.close()